In [1]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import pickle

from Extrapolability import get_sample_fromGP, get_sample_lack_feature, classifier_performance_update, get_sample_lack_features_mul
from utils import *

from xgboost import XGBClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF  
from sklearn.ensemble import RandomForestClassifier

In [3]:
info_path = '/Users/haoliu/Documents/Hao/Fordham/Cheminformatics/MD/Amphiphiles/info_new_molecules.csv'
df_info = pd.read_csv(info_path)
df_info['Name']

0                                              Decanal
1                                             Geraniol
2                                    Hexadecanoic acid
3                                     Myristoleic acid
4                                   Glycine octylester
5                               Perfluorooctanoic acid
6     1H,1H,2H,2H-Tridecafluorooctane-1-sulphonic acid
7    Heptadecafluorooctanesulfonic acid potassium salt
Name: Name, dtype: object

In [4]:
output_file = "selected_samples_20250603.pkl"

with open(output_file, 'rb') as f:
    loaded_list = pickle.load(f)

In [6]:
len(loaded_list)

10

In [7]:
high_samples0 = loaded_list[0]
low_samples0 = loaded_list[1]
high_samples1 = loaded_list[2]
low_samples1 = loaded_list[3]
high_samples2 = loaded_list[4]
low_samples2 = loaded_list[5]
high_samples3 = loaded_list[6]
low_samples3 = loaded_list[7]
high_samples4 = loaded_list[8]
low_samples4 = loaded_list[9]

In [8]:
datasets = [
    high_samples0, low_samples0, high_samples1, low_samples1,
    high_samples2, low_samples2, high_samples3, low_samples3,
    high_samples4, low_samples4
]

# Process all datasets
original_concentrations = [
    extract_original_concentrations(df, df_info) for df in datasets
]

In [9]:
original_concentrations[4]

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,Geraniol,Decanoate,Glycine octylester,Myristoleic acid,Hexadecanoic acid,Decanal,Glycerol monodecanoate
7,0.392659,4.681768,3.010242,4.383594,3.969481,0.729531,0.894575
845,0.162868,4.836453,2.300393,0.667583,1.856588,0.105522,0.953196
71,0.149448,4.730949,4.853574,2.493618,1.613148,0.327925,0.757829
400,3.137699,4.087244,4.424286,4.586383,1.545508,1.448555,0.690336
963,0.224956,1.777742,3.684302,1.351059,4.941847,0.863243,0.052686
388,2.184510,4.728892,3.561855,0.043183,4.002834,1.049585,0.717579
146,4.239865,4.696790,4.429779,1.338893,3.685971,1.168103,0.247552
769,0.016415,2.195020,1.498667,1.221674,4.655197,0.687730,0.819361
447,3.492349,3.231380,3.369240,4.318508,1.651624,0.201435,0.533453
303,1.472449,4.656800,1.113884,4.831586,2.309351,0.788797,0.121908


In [26]:
len(original_concentrations)

10

In [30]:
import joblib

def save_objects_joblib(filepath, **kwargs):
    """
    Save multiple Python objects using joblib.

    Parameters:
    - filepath: str — path to save the .pkl file.
    - **kwargs: any number of named Python objects to save.
    """
    joblib.dump(kwargs, filepath)

In [31]:
save_objects_joblib(
    "experimental_plan1_concentrations_20250611.joblib",
    original_concentrations=original_concentrations,
)

In [10]:
# Optional: descriptive names for each dataset
names = [
    "high_samples0", "low_samples0", "high_samples1", "low_samples1",
    "high_samples2", "low_samples2", "high_samples3", "low_samples3",
    "high_samples4", "low_samples4"
]

# Save with descriptive names
for name, df in zip(names, original_concentrations):
    filename = f"{name}_original_concentrations.csv"
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

Saved: high_samples0_original_concentrations.csv
Saved: low_samples0_original_concentrations.csv
Saved: high_samples1_original_concentrations.csv
Saved: low_samples1_original_concentrations.csv
Saved: high_samples2_original_concentrations.csv
Saved: low_samples2_original_concentrations.csv
Saved: high_samples3_original_concentrations.csv
Saved: low_samples3_original_concentrations.csv
Saved: high_samples4_original_concentrations.csv
Saved: low_samples4_original_concentrations.csv


In [34]:
import os

In [35]:
# Use more descriptive names for the files
save_dir = "../results/experimental_plan_fm_update/region/concentrations"

names = [
    "high_original_fm", "low_original_fm", "high_replace_smi1_fm", "low_replace_smi1_fm",
    "high_replace_smi5_fm", "low_replace_smi5_fm", "high_replace_dissmi1_fm", "low_replace_dissmi1_fm",
    "high_replace_dissmi3_fm", "low_replace_dissmi3_fm"
]

# Save with descriptive names
for name, df in zip(names, original_concentrations):
    filename = os.path.join(save_dir, f"{name}_original_concentrations.csv")
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

Saved: ../results/experimental_plan_fm_update/region/concentrations/high_original_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/low_original_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/high_replace_smi1_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/low_replace_smi1_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/high_replace_smi5_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/low_replace_smi5_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/high_replace_dissmi1_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/low_replace_dissmi1_fm_original_concentrations.csv
Saved: ../results/experimental_plan_fm_update/region/concentrations/hi

In [11]:
 # Generate dispense volumes

In [13]:
def get_dispense_volume(concentrations_df, unique_solutes, allow_zero=False):

    stocks = [[50, 10, 2], [50, 10, 2], [50, 10, 2], [50, 10, 2], [50, 10, 2], [15, 3], [10, 2]]
    valid_dispense_volumes = [[(4, 20), (4, 20), (0, 20)], [(4, 20), (4, 20), (0, 20)], [(4, 20), (4, 20), (0, 20)],
                            [(4, 20), (4, 20), (0, 20)], [(4, 20), (4, 20), (0, 20)], [(4, 20), (0, 20)], [(4, 20), (0, 20)]]

    # Total volume
    total_volume = 200

    # Initialize empty DataFrame to store dispense volumes
    column_names = []
    for i, (col, stock) in enumerate(zip(concentrations_df.columns, stocks), start=1):
        for j, concentration in enumerate(stock, start=1):
            column_names.append(f'{col} ({concentration} mM)')

    dispense_df = pd.DataFrame(columns=column_names)

    # Iterate through each row of concentrations dataframe
    for index, row in concentrations_df.iterrows():
        dispense_volumes = []
        
        # Calculate required volume for each stock solution
        for i, (conc, stock, valid_vol) in enumerate(zip(row, stocks, valid_dispense_volumes), start=1):
            total_mass = conc * total_volume
            volumes = [total_mass / s for s in stock]
            
            # Initialize a list to store volumes for this stock solution
            stock_dispense_volumes = []
            
            # Flag to track if a valid volume has been encountered
            valid_volume_found = False
            
            # Check all possible volumes within the valid dispense volume range
            for volume, vol_range in zip(volumes, valid_vol):
                if not valid_volume_found and vol_range[0] <= volume <= vol_range[1]:
                    stock_dispense_volumes.append(volume)
                    # Set the flag to True to indicate a valid volume has been found
                    valid_volume_found = True
                else:
                    stock_dispense_volumes.append(0)
            
            # Append the volumes for this stock solution to dispense_volumes
            dispense_volumes.extend(stock_dispense_volumes)
            
        # Append dispense volumes for this row to the dataframe
        dispense_df.loc[index] = dispense_volumes
        # Round all numbers in the DataFrame to one decimal place
        dispense_df = dispense_df.round(1)
        # Apply custom function to each element of the DataFrame
        # dispense_df = dispense_df.applymap(discretize)

    # Iterate over the rows and remove those where all columns for the same solute are zero
    if allow_zero:
        return dispense_df
    else:
        rows_to_remove = []
        for index, row in dispense_df.iterrows():
            all_zero = True
            for solute in unique_solutes:
                solute_cols = [col for col in dispense_df.columns if solute in col]
                if all(row[col] == 0 for col in solute_cols):
                    all_zero = all_zero and True
                    rows_to_remove.append(index)
                    # print(index)
                else:
                    all_zero = False

        # Remove the identified rows
        dispense_df = dispense_df.drop(rows_to_remove)
        return dispense_df


In [15]:
solutes = original_concentrations[0].columns
dispense_df = get_dispense_volume(original_concentrations[0], solutes, allow_zero=True)
len(dispense_df)

10

In [16]:
dispense_df

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,Decanoic acid (50 mM),Decanoic acid (10 mM),Decanoic acid (2 mM),Decanoate (50 mM),Decanoate (10 mM),Decanoate (2 mM),Decylamine (50 mM),Decylamine (10 mM),Decylamine (2 mM),Decyltrimethyl ammonium bromid (50 mM),Decyltrimethyl ammonium bromid (10 mM),Decyltrimethyl ammonium bromid (2 mM),Decyl sodium sulfate (50 mM),Decyl sodium sulfate (10 mM),Decyl sodium sulfate (2 mM),Decanol (15 mM),Decanol (3 mM),Glycerol monodecanoate (10 mM),Glycerol monodecanoate (2 mM)
1,14.9,0.0,0.0,7.7,0.0,0.0,10.2,0.0,0.0,0.0,0.0,3.8,18.7,0.0,0.0,15.3,0.0,0.0,15.9
408,0.0,9.7,0.0,0.0,0.0,7.2,0.0,0.0,6.7,0.0,7.4,0.0,14.9,0.0,0.0,0.0,19.7,18.6,0.0
400,12.6,0.0,0.0,16.3,0.0,0.0,17.7,0.0,0.0,18.3,0.0,0.0,6.2,0.0,0.0,19.3,0.0,13.8,0.0
250,6.4,0.0,0.0,0.0,9.4,0.0,0.0,0.0,2.2,0.0,10.8,0.0,19.0,0.0,0.0,14.5,0.0,18.9,0.0
143,6.6,0.0,0.0,15.1,0.0,0.0,10.9,0.0,0.0,13.0,0.0,0.0,9.0,0.0,0.0,11.8,0.0,16.0,0.0
162,9.6,0.0,0.0,0.0,0.0,14.6,14.4,0.0,0.0,5.7,0.0,0.0,5.5,0.0,0.0,9.6,0.0,15.4,0.0
843,6.6,0.0,0.0,0.0,5.2,0.0,0.0,19.2,0.0,0.0,0.0,19.4,8.2,0.0,0.0,6.7,0.0,14.3,0.0
131,17.9,0.0,0.0,11.7,0.0,0.0,0.0,5.7,0.0,15.0,0.0,0.0,5.6,0.0,0.0,16.9,0.0,14.8,0.0
150,15.2,0.0,0.0,6.3,0.0,0.0,17.5,0.0,0.0,12.7,0.0,0.0,19.9,0.0,0.0,5.1,0.0,14.0,0.0
614,4.7,0.0,0.0,7.6,0.0,0.0,0.0,0.0,15.5,17.5,0.0,0.0,11.5,0.0,0.0,9.5,0.0,15.5,0.0


In [18]:
solutes = original_concentrations[4].columns
dispense_df = get_dispense_volume(original_concentrations[4], solutes, allow_zero=True)
dispense_df

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,Geraniol (50 mM),Geraniol (10 mM),Geraniol (2 mM),Decanoate (50 mM),Decanoate (10 mM),Decanoate (2 mM),Glycine octylester (50 mM),Glycine octylester (10 mM),Glycine octylester (2 mM),Myristoleic acid (50 mM),Myristoleic acid (10 mM),Myristoleic acid (2 mM),Hexadecanoic acid (50 mM),Hexadecanoic acid (10 mM),Hexadecanoic acid (2 mM),Decanal (15 mM),Decanal (3 mM),Glycerol monodecanoate (10 mM),Glycerol monodecanoate (2 mM)
7,0.0,7.9,0.0,18.7,0.0,0.0,12.0,0.0,0.0,17.5,0.0,0.0,15.9,0.0,0.0,9.7,0.0,17.9,0.0
845,0.0,0.0,16.3,19.3,0.0,0.0,9.2,0.0,0.0,0.0,13.4,0.0,7.4,0.0,0.0,0.0,7.0,19.1,0.0
71,0.0,0.0,14.9,18.9,0.0,0.0,19.4,0.0,0.0,10.0,0.0,0.0,6.5,0.0,0.0,4.4,0.0,15.2,0.0
400,12.6,0.0,0.0,16.3,0.0,0.0,17.7,0.0,0.0,18.3,0.0,0.0,6.2,0.0,0.0,19.3,0.0,13.8,0.0
963,0.0,4.5,0.0,7.1,0.0,0.0,14.7,0.0,0.0,5.4,0.0,0.0,19.8,0.0,0.0,11.5,0.0,0.0,5.3
388,8.7,0.0,0.0,18.9,0.0,0.0,14.2,0.0,0.0,0.0,0.0,4.3,16.0,0.0,0.0,14.0,0.0,14.4,0.0
146,17.0,0.0,0.0,18.8,0.0,0.0,17.7,0.0,0.0,5.4,0.0,0.0,14.7,0.0,0.0,15.6,0.0,5.0,0.0
769,0.0,0.0,1.6,8.8,0.0,0.0,6.0,0.0,0.0,4.9,0.0,0.0,18.6,0.0,0.0,9.2,0.0,16.4,0.0
447,14.0,0.0,0.0,12.9,0.0,0.0,13.5,0.0,0.0,17.3,0.0,0.0,6.6,0.0,0.0,0.0,13.4,10.7,0.0
303,5.9,0.0,0.0,18.6,0.0,0.0,4.5,0.0,0.0,19.3,0.0,0.0,9.2,0.0,0.0,10.5,0.0,0.0,12.2


In [19]:
# List of base names for saving the dispense volumes
filenames = [
    "high_samples0", "low_samples0", "high_samples1", "low_samples1",
    "high_samples2", "low_samples2", "high_samples3", "low_samples3",
    "high_samples4", "low_samples4"
]

# Loop over all original concentration DataFrames
for i, df in enumerate(original_concentrations):
    solutes = df.columns
    dispense_df = get_dispense_volume(df, solutes, allow_zero=True)
    dispense_df.to_csv(f"{filenames[i]}_dispense_volumes.csv", index=False)
    print(f"Saved: {filenames[i]}_dispense_volumes.csv")


Saved: high_samples0_dispense_volumes.csv
Saved: low_samples0_dispense_volumes.csv
Saved: high_samples1_dispense_volumes.csv
Saved: low_samples1_dispense_volumes.csv
Saved: high_samples2_dispense_volumes.csv
Saved: low_samples2_dispense_volumes.csv
Saved: high_samples3_dispense_volumes.csv
Saved: low_samples3_dispense_volumes.csv
Saved: high_samples4_dispense_volumes.csv
Saved: low_samples4_dispense_volumes.csv


In [36]:
# Use more descriptive names

In [37]:
# List of base names for saving the dispense volumes
save_dir = "../results/experimental_plan_fm_update/region/dispense_volume"

filenames = [
    "high_original_fm", "low_original_fm", "high_replace_smi1_fm", "low_replace_smi1_fm",
    "high_replace_smi5_fm", "low_replace_smi5_fm", "high_replace_dissmi1_fm", "low_replace_dissmi1_fm",
    "high_replace_dissmi3_fm", "low_replace_dissmi3_fm"
]
# filenames = [
#     "high_samples0", "low_samples0", "high_samples1", "low_samples1",
#     "high_samples2", "low_samples2", "high_samples3", "low_samples3",
#     "high_samples4", "low_samples4"
# ]

# Loop over all original concentration DataFrames
for i, df in enumerate(original_concentrations):
    solutes = df.columns
    dispense_df = get_dispense_volume(df, solutes, allow_zero=True)
    filename = os.path.join(save_dir, f"{filenames[i]}_dispense_volumes.csv")
    dispense_df.to_csv(filename, index=False)
    print(f"Saved: {filenames[i]}_dispense_volumes.csv")


Saved: high_original_fm_dispense_volumes.csv
Saved: low_original_fm_dispense_volumes.csv
Saved: high_replace_smi1_fm_dispense_volumes.csv
Saved: low_replace_smi1_fm_dispense_volumes.csv
Saved: high_replace_smi5_fm_dispense_volumes.csv
Saved: low_replace_smi5_fm_dispense_volumes.csv
Saved: high_replace_dissmi1_fm_dispense_volumes.csv
Saved: low_replace_dissmi1_fm_dispense_volumes.csv
Saved: high_replace_dissmi3_fm_dispense_volumes.csv
Saved: low_replace_dissmi3_fm_dispense_volumes.csv
